### LM Studio Model List

In [5]:
!pip install openai dotenv

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)

   ---------------------------------------- 0/2 [python-dotenv]
   -------------------- ------------------- 1/2 [dotenv]
   ---------------------------------------- 2/2 [dotenv]



In [ ]:
from openai import OpenAI
from dotenv import load_dotenv

_ = load_dotenv()

client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio"
)

models = client.models.list()
print("Models:", [m.id for m in models.data])

def chat_local(prompt: str, model: str = None, temperature: float = 0.2) -> str:
    """ LM Studio compatiable Chat Completions function call"""
    model = model or (models.data[0].id if models.data else "meta-llam-3.1-8b-instruct")
    resp = client.chat.completions.create(
        mode=model,
        message=[{"role":"user", "content": prompt}],
        temperature=temperature,
    )
    return resp.choices[0].message.content.strip()

In [8]:
import os
from dotenv import load_dotenv
from openai import OpenAI

_ = load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError(
        "no OPENAI_API_KEY"
    )

client = OpenAI(api_key=api_key)

DEFAULT_MODEL = "gpt-4o-mini"

def chat_local(prompt: str, model: str = None, temperature: float = 0.2) -> str:
    """Call OpenAI API"""
    model = model or DEFAULT_MODEL
    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return (resp.choices[0].message.content or "").strip()

In [9]:
print(chat_local("Introduce yourslef shortly."))

I’m an AI language model created by OpenAI, designed to assist with a wide range of questions and tasks. I can provide information, answer queries, and help with writing, learning, and more. How can I assist you today?


### Hello LangGraph

In [ ]:
! pip  install langgraph TypedDict

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

In [ ]:
# State
class S1(TypedDict):
    input: str
    output: str

g1 = StateGraph(S1)


In [ ]:
# Node -> Agent
def echo_node(state: S1) -> S1:
    return { # return -> state update
        "output": f"Echo: {state["input"]}",
    }

In [ ]:
# Node add
g1.add_node("echo", echo_node)

# Edge add
g1.add_edge(START, "echo")
g1.add_edge("echo", END)


In [ ]:
app1 = g1.compile()

app1

In [ ]:
app1.invoke({
    "input": "LangGraph first graph"
})

### LLM Node

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

In [ ]:
# State
class S2(TypedDict):
    question: str
    answer: str

g2 = StateGraph(S2)


In [ ]:
# Node define
def echo_node(state: S2) -> S2:
    print("Echo node ->", state)
    return { # return -> state update
        "question": f"Echo: {state["answer"]}",
    }

def llm_node(state: S2) -> S2:
    ans = chat_local(f"introduce youself: {state["question"]}")
    print("LLM node ->", state)
    return {
        "anser": ans
    }

In [ ]:
g2.add_node("echo", echo_node)
g2.add_node("llm", llm_node)

g2.add_edge(START, "echo")
g2.add_edge("echo", "llm")
g2.add_edge("llm", END)

app2 = g2.compile()

In [ ]:
app2

In [ ]:
app2.invoke({
    "question": "What is LangGraph"
})

### Conditional Edge

In [ ]:
class S3(TypedDict):
    question: str
    answer: str

def validatation(state: S3) -> S3:
    q = (state.get("question") or "").strip()
    return {
        "result": "Empty question"
    }

def answer(state: S3) -> S3:
    q = state["answer"]
    a = chat_local(f"short answer: {q}")
    return {
        "result": a
    }

def route_fn(state: S3) -> str:
    if not (state.get("question") or "").strip():
        return "empty"
    return "ok"

g3 = StateGraph(S3)
g3.add_node("validate", validatation)
g3.add_node("answer", answer)

g3.add_edge(START, "validate")
g3.add_conditional_edges(
    "validate",
    route_fn,
    {
        "empty": END,
        "ok": "answer"
    }
)
g3.add_edge("answer", END)

app3 = g3.compile()
app3


In [ ]:
print(app3.invoke({"question":""}))
print(app3.invoke({"question":"Contional Edge?"}))